In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size



In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"url: {self.config.source_URL}")
                logger.info(f"filename: {self.config.local_data_file}")
                logger.info(f"{filename} downloaded with the following info:\n{headers}")
            except Exception as e:
                logger.error(f"Failed to download file. Error: {e}")
                raise e
        else:
            size = get_size(Path(self.config.local_data_file))
            logger.info(f"File already exists. Size: {size}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to {unzip_path}")
        except zipfile.BadZipFile as e:
            logger.error(f"Invalid ZIP file. Error: {e}")
            raise e
        

        


In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-09 18:19:21,506]: INFO: common: yaml file: config/config.yaml loaded successfully
[2025-04-09 18:19:21,507]: INFO: common: yaml file: params.yaml loaded successfully
[2025-04-09 18:19:23,988]: INFO: 2511603307: url: https://github.com/supriya-nkamble/Text-Summarizer/raw/main/data/samsumdata.zip
[2025-04-09 18:19:23,989]: INFO: 2511603307: filename: artifacts/data_ingestion/file.zip
[2025-04-09 18:19:23,993]: INFO: 2511603307: artifacts/data_ingestion/file.zip downloaded with the following info:
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AE2A:5ABA6:25D072:6E8EE2:67F625F9
Accept-Ranges: bytes
Date: Wed, 09 Apr 2025 08:19:22 